<h1><center>Laboratorio 6: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Patricio Ortiz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Sebastián Aguirre
- Nombre de alumno 2: Cristóbal Gallardo


### **Link de repositorio de GitHub:** `https://github.com/cristobalgallardo/MDS7202`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- Fecha de entrega: 09/06/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.2 MB 24.5 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0


In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = '/content/drive/MyDrive/Lab_MDS/Laboratorios/Lab6'
except: 
    print('Ignorando conexión drive-colab')

Mounted at /content/drive


# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la farbicación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito. 
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [3]:
df_retail = pd.read_pickle(f"{path}/online_retail_II_cleaned.pickle")
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 1.2 Creación de nuevas Caracteristicas [2 Puntos] 

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.
- **Recency (R)**: Indica la actualidad de la interacción de un cliente con la empresa, y da información sobre la tendencia a repetir la compra. Se define como: 
$$Recency(n)=\dfrac{1}{n} \sum^n_{i=1} date\_diff(t_{fecha final}, t_{m-i+1})$$

 Donde $date\_diff$ representa la diferencia en días entre la fecha de finalización del periodo de observación ($t_{fecha final}$), y la fecha de una visita del cliente cercana a $t_{fecha final}$, $t_{m-i+1}; t_{m}$ es la última visita del cliente; y n es el número de visitas recientes del cliente consideradas.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente. 

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.
 

$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**Nota:** Para la $fechafinal$ utilice la fecha máxima del dataset más 1 día.

**Ejemplo de Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [4]:
df = df_retail.copy()
df = df.dropna()

# Pasar a día
#df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'].dt.date)


# Dropear duplicados
df_c  = df[['Customer ID','InvoiceDate','Invoice']].drop_duplicates().copy()

# Sacar largo
result = df_c.groupby('Customer ID')['InvoiceDate'].agg(['min', 'max'])
result['Length'] = (result['max'] - result['min']).dt.days

# Sacar frequencia
result['Frequency'] = df_c.groupby('Customer ID')['Invoice'].count()

# Ultimo registro + 1 día
tfinal = result['max'].max() + datetime.timedelta(days=1)

# Promedio de diferencia entre ultimo registros y ultimos n registros de cada comprador
n = 1
result['Recency'] =  df_c.groupby('Customer ID').agg({'InvoiceDate': lambda x: ((tfinal - x.nlargest(n)).dt.days).mean() })

# Suma(Precio * Cantidad) por id de comprador
df['Sales'] =  df['Price'] * df['Quantity']
result['Monetary'] = df.groupby('Customer ID')['Sales'].sum()
# Dividir por frecuencia
result['Monetary'] = result['Monetary']/result['Frequency']

# Periodicidad
df_c = df_c.sort_values(['Customer ID','InvoiceDate'])
df_c['diff'] = df_c.groupby(['Customer ID'])['InvoiceDate'].transform(lambda x: x.diff().dt.days)
result['Periodicity'] = df_c.groupby(['Customer ID'])['diff'].std().fillna(0.0)

In [8]:
result

,min,max,Length,Frequency,Recency,Monetary,Periodicity
Customer ID,,,,,,,
12346.0,2009-12-14 08:34:00,2010-06-28 13:53:00,196,11,165.0,33.896364,36.659999
12347.0,2010-10-31 14:20:00,2010-12-07 14:57:00,37,2,3.0,661.660000,0.000000
12348.0,2010-09-27 14:59:00,2010-09-27 14:59:00,0,1,74.0,222.160000,0.000000
12349.0,2010-04-29 13:20:00,2010-10-28 08:23:00,181,3,43.0,890.380000,101.823376
12351.0,2010-11-29 15:23:00,2010-11-29 15:23:00,0,1,11.0,300.930000,0.000000
...,...,...,...,...,...,...,...
18283.0,2010-02-19 17:16:00,2010-11-22 15:30:00,275,6,18.0,103.228333,61.832839
18284.0,2010-10-04 11:33:00,2010-10-04 11:33:00,0,1,67.0,461.680000,0.000000
18285.0,2010-02-17 10:24:00,2010-02-17 10:24:00,0,1,296.0,427.000000,0.000000


In [5]:
def custom_features(dataframe_in):
  """
  Entreega los campos:
  Length (L): Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

  Recency (R): Indica la actualidad de la interacción de un cliente con la empresa, y da información sobre la tendencia a repetir la compra. Se define como:
  Recency(n)=1n∑i=1ndate_diff(tfechafinal,tm−i+1) 

  Donde  date_diff  representa la diferencia en días entre la fecha de finalización del periodo de observación ( tfechafinal ), y la fecha de una visita del cliente cercana a  tfechafinal ,  tm−i+1;tm  es la última visita del cliente; y n es el número de visitas recientes del cliente consideradas.

  Monetary (M): El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

  Frequency (F): Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente.

  Periodicity (P): Representa si los clientes visitan las tiendas con regularidad.

  Periodicity(n)=std(IVT1,...,IVTn)

  para cada customer_id
  """

  df = df_retail.copy()
  df = df.dropna()

  # Dropear duplicados de la misma transacción 
  df_c  = df[['Customer ID','InvoiceDate','Invoice']].drop_duplicates().copy()
  # Sacar largo
  result = df_c.groupby('Customer ID')['InvoiceDate'].agg(['min', 'max'])
  result['Length'] = (result['max'] - result['min']).dt.days

  # Sacar frequencia
  result['Frequency'] = df_c.groupby('Customer ID')['Invoice'].count()

  # Ultimo registro + 1 día
  tfinal = result['max'].max() + datetime.timedelta(days=1)

  # Promedio de diferencia entre ultimo registros y ultimos n registros de cada comprador
  n = 1
  result['Recency'] =  df_c.groupby('Customer ID').agg({'InvoiceDate': lambda x: ((tfinal - x.nlargest(n)).dt.days).mean() })

  # Suma(Precio * Cantidad) por id de comprador
  df['Sales'] =  df['Price'] * df['Quantity']
  result['Monetary'] = df.groupby('Customer ID')['Sales'].sum()
  # Dividir por frecuencia
  result['Monetary'] = result['Monetary']/result['Frequency']

  # Periodicidad
  df_c = df_c.sort_values(['Customer ID','InvoiceDate'])
  df_c['diff'] = df_c.groupby(['Customer ID'])['InvoiceDate'].transform(lambda x: x.diff().dt.days)
  # Si no tiene periodicidad se remplaza por 0
  result['Periodicity'] = df_c.groupby(['Customer ID'])['diff'].std().fillna(0.0)
  # Periodicidad
  result = result.drop(columns= ['max', 'min'])
  result = result.reset_index()
  return result



In [6]:
custom_features(df_retail)

,Customer ID,Length,Frequency,Recency,Monetary,Periodicity
0,12346.0,196,11,165.0,33.896364,36.659999
1,12347.0,37,2,3.0,661.660000,0.000000
2,12348.0,0,1,74.0,222.160000,0.000000
3,12349.0,181,3,43.0,890.380000,101.823376
4,12351.0,0,1,11.0,300.930000,0.000000
...,...,...,...,...,...,...
4309,18283.0,275,6,18.0,103.228333,61.832839
4310,18284.0,0,1,67.0,461.680000,0.000000
4311,18285.0,0,1,296.0,427.000000,0.000000
4312,18286.0,247,2,112.0,648.215000,0.000000


**Resúesta de Pauta:**

## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [7]:
class MinMax(BaseEstimator, TransformerMixin):
    def fit(self,X,y=None):
      self.max_ = X.max(axis=0)
      self.min_ = X.min(axis=0)
      return self
    
    def transform(self,X):
      # MinMax = (x-min)/(max - min) 
      Xmm = (X- self.min_)/(self.max_- self.min_)
      return Xmm
  
    def inverse_transform(self, X):
      # x = MinMax * (max - min) + min
      Xn = X*(self.max_- self.min_) + self.min_
      return Xn

### 1.3.2 `T-SNA` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo. 

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior. 
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo T-sne de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [9]:
LRMFP_features = FunctionTransformer(custom_features)
# MinMax
minmax_scaler = ColumnTransformer(transformers = [('MinMax', MinMax(),  ['Length','Recency','Frequency','Monetary','Periodicity'] ) ])
# Pipeline [LRMFP -> MINMAX -> TSNE-2D]
tsne_pipeline = Pipeline([ ('LRMFP', LRMFP_features), ('MinMax', minmax_scaler), ('TSNE', TSNE(n_components=2, random_state=0,init = 'pca', learning_rate = 'auto'))])

In [10]:
output_tsne = tsne_pipeline.fit_transform(df_retail)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:986: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  FutureWarning,


In [11]:
features = custom_features(df_retail)
features[['x_tsne', 'y_tsne']] = output_tsne
features[['Length','Recency','Frequency','Monetary','Periodicity']] = minmax_scaler.transform(features[['Length','Recency','Frequency','Monetary','Periodicity']])
features

,Customer ID,Length,Frequency,Recency,Monetary,Periodicity,x_tsne,y_tsne
0,12346.0,0.525469,0.048780,0.439678,0.002853,0.143615,-4.774650,13.862163
1,12347.0,0.099196,0.004878,0.005362,0.055691,0.000000,0.435028,-40.628464
2,12348.0,0.000000,0.000000,0.195710,0.018699,0.000000,-46.669735,-37.842754
3,12349.0,0.485255,0.009756,0.112601,0.074943,0.398892,14.394070,26.588839
4,12351.0,0.000000,0.000000,0.026810,0.025329,0.000000,7.927671,-58.569885
...,...,...,...,...,...,...,...,...
4309,18283.0,0.737265,0.024390,0.045576,0.008689,0.242229,30.758282,16.659546
4310,18284.0,0.000000,0.000000,0.176944,0.038859,0.000000,-39.582951,-43.295151
4311,18285.0,0.000000,0.000000,0.790885,0.035940,0.000000,-13.175411,47.740292
4312,18286.0,0.662198,0.004878,0.297587,0.054560,0.000000,10.314581,3.793167


In [12]:
def plot_component(features, color_ft):
    fig = px.scatter(
        features,
        x='x_tsne',
        y='y_tsne',
        color=color_ft,
    )
    return fig

In [13]:
plot_component(features, "Length")

In [14]:
plot_component(features, "Recency")

In [15]:
plot_component(features, "Monetary")

In [16]:
plot_component(features, "Periodicity")

### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero permutando el algoritmo de reducción de dimensionalidad por k-means.

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

In [17]:
pipeline_kmeans = Pipeline([('LRMFP',LRMFP_features),('MinMax', minmax_scaler),
                  ('Kmeans', KMeans(random_state=0))])



inertias = []

for k in range(20):
  print(f'Fitting {k+1} clusters')
  # Reutilizar transformación después de la 1era vez
  if k == 0:
    cluster_features = pipeline_kmeans['LRMFP'].fit_transform(df_retail)
    cluster_features = pipeline_kmeans['MinMax'].fit_transform(cluster_features)
  # Actualizar k clusters
  pipeline_kmeans.set_params(Kmeans__n_clusters= k+1)
  pipeline_kmeans['Kmeans'].fit(cluster_features)
  # Guardar inercias
  inertias.append(pipeline_kmeans.named_steps['Kmeans'].inertia_)
px.line(x = np.arange(1, 21), y = inertias, labels={'x': 'k clusters', 'y': 'inertia'} )

Fitting 1 clusters
Fitting 2 clusters
Fitting 3 clusters
Fitting 4 clusters
Fitting 5 clusters
Fitting 6 clusters
Fitting 7 clusters
Fitting 8 clusters
Fitting 9 clusters
Fitting 10 clusters
Fitting 11 clusters
Fitting 12 clusters
Fitting 13 clusters
Fitting 14 clusters
Fitting 15 clusters
Fitting 16 clusters
Fitting 17 clusters
Fitting 18 clusters
Fitting 19 clusters
Fitting 20 clusters


En base al resultado se escogen 4 clusters, ya que este corresponde al valor del codo del gráfico.

#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:**

In [35]:
# Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.
k = 4
pipeline_kmeans.set_params(Kmeans__n_clusters= k)
cluster_labels = pipeline_kmeans['Kmeans'].fit_predict(cluster_features)

In [36]:
df_clusters = custom_features(df_retail)
df_clusters['cluster'] = cluster_labels

In [42]:
means = df_clusters.groupby(by='cluster').mean()
counts = df_clusters.groupby(by='cluster').count()['Length']
means['Count'] = counts
means

,Length,Frequency,Recency,Monetary,Periodicity,Count
cluster,,,,,,
0,17.349765,1.705790,52.108764,369.825669,2.125616,1278
1,318.460854,10.168149,23.920819,409.309579,46.678549,1124
2,181.530000,4.273000,67.416000,365.542215,33.320335,1000
3,17.707237,1.500000,255.303728,352.289942,2.216331,912


Sobre las 4 agrupaciones, se puede ver que monetariamente son todas similares, siendo la agrupación 1 la que gasta más en promedio y la agrupación 3 la que gasta menos. Los grupos 0 y 3 son bastante parecidos en terminos de Length, Frequency, pero el grupo 3 tiene una Recency 5 veces mayor. 

El grupo 1 es el mejor grupo en terminos monetarios, length, frecuencia y periodicidad, este grupo corresponde a las personas que compran seguido y llevan mucho tiempo comprando en la tienda y en general gastan mas dinero en cada compra. 

El grupo 2 corresponde a clientes no tan frecuentes como en 1, pero que repiten las compras más que el grupo 1. 

**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

In [21]:
features['cluster'] = cluster_labels

In [22]:
plot_component(features, 'cluster')

De la figura se puede ver que los clusters en general están bien separados, salvo los clusters naranjo y morado, donde se puede ver que parte por la mitad a una agrupación. 

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab** solo para tareas. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>